In [1]:
import datasets
from datasets import concatenate_datasets, load_dataset, DatasetDict
import torch
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import random
import numpy as np
from typing import Dict, List, Optional, Iterator, Callable, Union, Tuple
import re
from huggingface_hub import login
from transformers import AutoTokenizer, CLIPTextModel, CLIPTokenizer, CLIPTextModelWithProjection, CLIPProcessor, CLIPModel
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoProcessor
from sklearn.model_selection import train_test_split
import clip
from PIL import Image
import torch.nn as nn
from torch import nn, optim

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-14 22:06:18.996052: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 22:06:19.045625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 22:06:19.045649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 22:06:19.047024: E external/l

In [2]:
# 데이터셋 로드
ds = datasets.load_dataset('kimdeoldeol/COCO_dataset')
# 데이터 소스 경로
source_path = '/workspace/cache_kkuldanji/747f1772-e403-4641-a3eb-d57c6a8cbaea/source/'

In [3]:
# clip 모델로드
model = AutoModel.from_pretrained('laion/CLIP-ViT-bigG-14-laion2B-39B-b160k')
tokenizer = AutoTokenizer.from_pretrained('laion/CLIP-ViT-bigG-14-laion2B-39B-b160k')
processor = AutoProcessor.from_pretrained('laion/CLIP-ViT-bigG-14-laion2B-39B-b160k')

/home/trevor/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:830: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1280)
      (position_embedding): Embedding(77, 1280)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-31): 32 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (layer_norm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1280, out_features=5120, bias=True)
            (fc2): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (layer_norm2): LayerNorm((1280,), e

In [5]:
# Define a custom dataset 데이터 전처리 함수
class image_title_dataset():
    def __init__(self, list_image_path,list_txt,tokenizer,processor):
        # Initialize image paths and corresponding texts
        self.image_path = list_image_path
        
        # self.title  = tokenizer(list_txt)
        self.title  = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        
        image = processor(images=Image.open(self.image_path[idx]),return_tensors="pt").pixel_values.squeeze()
        title = self.title[idx]
        return image, title

In [6]:
# use your own data
data = []
for i in ds['train']:
    data.append(i)
    
train_img_paths, test_img_paths = train_test_split(data, test_size=0.1, random_state=42)

pre_train_list = []
pre_test_list = []
train_list_image_path = []
train_list_txt = []
test_list_image_path = []
test_list_txt = []

In [7]:
len(train_img_paths)

110030

In [8]:
len(test_img_paths)

12226

In [9]:
for item in tqdm(train_img_paths):
    image_path = source_path + item['source'].split('/')[-1]
    
    for i in item['caption_ko']:
        caption = i
        tokens = processor.tokenizer.tokenize(caption)
        num_token = len(tokens)
        
        if num_token <=75: 
            data_dict={"source":image_path,"caption":caption}
            pre_train_list.append(data_dict)

100%|██████████| 110030/110030 [01:02<00:00, 1755.84it/s]


In [10]:
len(pre_train_list)

545170

In [11]:
pre_train_list[0]

{'source': '/workspace/cache_kkuldanji/747f1772-e403-4641-a3eb-d57c6a8cbaea/source/COCO_train2014_000000337886.jpg',
 'caption': '시내에 오토바이 경찰이 죽 늘어서 있다'}

In [12]:
df = pd.DataFrame(pre_train_list)
df_no_duplicates = df.drop_duplicates(subset='caption')            

In [13]:
train_list_image_path = list(df_no_duplicates['source'])
train_list_txt = list(df_no_duplicates['caption'])

print(len(train_list_image_path))
print(len(train_list_txt))

511206
511206


In [14]:
for item in tqdm(test_img_paths):
    image_path = source_path + item['source'].split('/')[-1]
    
    for i in item['caption_ko']:
        caption = i
        tokens = processor.tokenizer.tokenize(caption)
        num_token = len(tokens)
        
        if num_token <=75: 
            data_dict={"source":image_path,"caption":caption}
            pre_test_list.append(data_dict)

100%|██████████| 12226/12226 [00:06<00:00, 1750.63it/s]


In [15]:
df = pd.DataFrame(pre_test_list)
df_no_duplicates = df.drop_duplicates(subset='caption')

In [16]:
test_list_image_path = list(df_no_duplicates['source'])
test_list_txt = list(df_no_duplicates['caption'])

print(len(test_list_image_path))
print(len(test_list_txt))

59159
59159


In [17]:
train_dataset = image_title_dataset(train_list_image_path, train_list_txt,tokenizer,processor)
test_dataset = image_title_dataset(test_list_image_path, test_list_txt,tokenizer,processor)

In [18]:
BATCH_SIZE = 16
EPOCH = 1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [19]:
# Function to convert model's parameters to FP32 format
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

In [20]:
if device == "cpu":
    model.float()

In [21]:
# Prepare the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5,betas=(0.9,0.999),eps=1e-6)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [22]:
# Specify the loss function
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

In [23]:
# Train the model
num_epochs = EPOCH

best_te_loss = 1e5
best_ep = -1

for epoch in range(num_epochs):
    print(f"running epoch {num_epochs}")
    step = 0
    tr_loss = 0
    
    model.train()
    train_pbar = tqdm(train_dataloader, total=len(train_dataloader))
    
    for batch in train_pbar:
        step += 1
        optimizer.zero_grad()
        

        # images,texts = batch
        texts,images = batch
        
        images= images.to(device)
        texts = texts.to(device)

        # Forward pass
        output = model(images, texts)
        logits_per_image = output.logits_per_image
        logits_per_text = output.logits_per_text

        # Compute loss
        # ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        
   
        # Backward pass
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
            
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            # clip.model.convert_weights(model)

        # pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")
        # pbar.set_description(f"train : {total_loss.item()}",refresh=True)
        train_pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
        
        tr_loss /= step
    
    
    step=0
    te_loss=0
    
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, total=len(test_dataloader))
        
        for batch in test_pbar:
            step +=1
            texts,images  = batch
            images = images.to(device)
            texts = texts.to(device)
            
            output = model(images, texts)
            logits_per_image = output.logits_per_image
            logits_per_text = output.logits_per_text
            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            
            total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
            
            te_loss /= step
        
        
        if te_loss < best_te_loss:
            best_te_loss = te_loss
            best_ep = epoch
            # torch.save(model.state_dict(), 'model/eval_test.pt')
            
            model.save_pretrained(f'/mnt/share/bcm/trevor/stable_diffusion/test_model_{epoch}')
        print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")

running epoch 1, best test loss 100000.0 after epoch -1


test batchCE: 0.00013090771972201765: 100%|██████████| 3698/3698 [26:31<00:00,  2.32it/s]


epoch 0, tr_loss 9.294714396925556e-08, te_loss 4.1360564738313094e-08
